<a href="https://colab.research.google.com/github/falawar7/AAI_635O/blob/main/Week3/FE_DL_Experimenting_With_NCF_Based_Recommendation_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install recommenders[gpu] -f https://download.pytorch.org/whl/cu111/torch_stable.html

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import sys
import pandas as pd
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
#from recommenders.datasets import movielens
from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets.python_splitters import python_chrono_split,python_stratified_split
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k,
                                                     recall_at_k, get_top_k_items)

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
Pandas version: 2.2.2
Tensorflow version: 2.19.0


In [ ]:
!pip uninstall numpy
!pip install numpy
!pip install --upgrade tensorflow

Found existing installation: numpy 2.2.5
Uninstalling numpy-2.2.5:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/numpy-config
    /usr/local/lib/python3.11/dist-packages/numpy-2.2.5.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1-0352e75f.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99-934c22de.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libscipy_openblas64_-6bb31eeb.so
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? y
  Successfully uninstalled numpy-2.2.5
  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.11.0 re

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
!pip install recommenders[gpu] -f https://download.pytorch.org/whl/cu111/torch_stable.html

import sys
import pandas as pd
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
#from recommenders.datasets import movielens
from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets.python_splitters import python_chrono_split,python_stratified_split
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k,
                                                     recall_at_k, get_top_k_items)

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

Looking in links: https://download.pytorch.org/whl/cu111/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 17.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.2/264.2 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 2.1 MB/s eta 0:00:00
   ━━━━

ImportError: cannot import name 'disable_resource_variables' from 'tensorflow.python.ops.variable_scope' (/usr/local/lib/python3.11/dist-packages/tensorflow/python/ops/variable_scope.py)

In [ ]:
# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
#MOVIELENS_DATA_SIZE = '100k'

# Model parameters
EPOCHS = 50
BATCH_SIZE = 256

SEED = 42

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##1. Load dataset

In [ ]:
df = pd.read_excel('/content/drive/Othercomputers/THINKPAD-T14/AI/Reommender_System/Week2/Lesson2_From_Course/Rec_sys_data.xlsx')

In [ ]:
df= df[['CustomerID','StockCode','Quantity','DeliveryDate']]

In [ ]:
df["StockCode"] = df["StockCode"].apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna()

In [ ]:
df = df.dropna()
print(df.shape)
df

(246706, 4)


,CustomerID,StockCode,Quantity,DeliveryDate
1,17850,71053.0,6,2010-12-02 08:26:00
2,17850,21730.0,6,2010-12-03 08:26:00
4,17850,22752.0,2,2010-12-04 08:26:00
7,17850,22633.0,6,2010-12-04 08:28:00
8,17850,22632.0,6,2010-12-03 08:28:00
...,...,...,...,...
272399,15249,23399.0,12,2011-10-08 11:37:00
272400,15249,22727.0,4,2011-10-08 11:37:00
272401,15249,23434.0,12,2011-10-08 11:37:00
272402,15249,23340.0,12,2011-10-07 11:37:00


In [ ]:
#header=["userID", "itemID", "rating", "timestamp"]

df = df.rename(columns={

    'CustomerID':"userID",'StockCode':"itemID",'Quantity':"rating",'DeliveryDate':"timestamp"

})

df["userID"] = df["userID"].astype(int)
df["itemID"] = df["itemID"].astype(int)

##2. Split the data using the Spark chronological splitter provided in utilities

In [ ]:
train, test = python_chrono_split(df, 0.75)

In [ ]:
train_file = "./train.csv"
test_file = "./test.csv"
train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185041 entries, 37126 to 137323
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   userID     185041 non-null  int64         
 1   itemID     185041 non-null  int64         
 2   rating     185041 non-null  int64         
 3   timestamp  185041 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 7.1 MB


##3. Train the NCF model on the training data, and get the top-k recommendations for our testing data
NCF accepts implicit feedback and generates prospensity of items to be recommended to users in the scale of 0 to 1. A recommended item list can then be generated based on the scores. Note that this quickstart notebook is using a smaller number of epochs to reduce time for training. As a consequence, the model performance will be slighlty deteriorated.

In [ ]:
data = NCFDataset(train_file=train_file, test_file=test_file, seed=SEED)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/li

In [ ]:
model = NCF (
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

/usr/local/lib/python3.11/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [ ]:
with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time))

Took 1203.4774 seconds for training.


In [ ]:
with Timer() as test_time:
    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item)
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time))

Took 23.4300 seconds for prediction.


##4. Evaluate how well NCF performs
The ranking metrics are used for evaluation.

In [ ]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.031644
NDCG:	0.072834
Precision@K:	0.049600
Recall@K:	0.057350


In [ ]:
df_order = pd.read_excel('/content/drive/Othercomputers/THINKPAD-T14/AI/Reommender_System/Week2/Lesson2_From_Course/Rec_sys_data.xlsx', 'order')
df_customer = pd.read_excel('/content/drive/Othercomputers/THINKPAD-T14/AI/Reommender_System/Week2/Lesson2_From_Course/Rec_sys_data.xlsx', 'customer')
df_product = pd.read_excel('/content/drive/Othercomputers/THINKPAD-T14/AI/Reommender_System/Week2/Lesson2_From_Course/Rec_sys_data.xlsx', 'product')

In [ ]:
all_predictions[all_predictions['userID']==12346].nlargest(5,'prediction')

,userID,itemID,timestamp,prediction
1918,12346,23167,NaT,0.948258
1925,12346,23174,NaT,0.944124
1924,12346,23173,NaT,0.936659
1921,12346,23170,NaT,0.934766
1472,12346,22698,NaT,0.924984


In [ ]:
all_predictions = all_predictions[['userID','itemID','prediction']]

In [ ]:
all_predictions = all_predictions.rename(columns={

    "userID":'CustomerID',"itemID":'StockCode',"rating":'Quantity','prediction':'probability'

})

In [ ]:
def recommend_product(customer_id):

  print(" \n---------- Top 5 Bought StockCodes -----------\n")


  print(df_order[df_order['CustomerID']==customer_id][['CustomerID','StockCode','Quantity']].nlargest(5,'Quantity'))

  top_5_bought = df_order[df_order['CustomerID']==customer_id][['CustomerID','StockCode','Quantity']].nlargest(5,'Quantity')

  print('\n-------Product Name of bought StockCodes ------\n')

  print(df_product[df_product.StockCode.isin(top_5_bought.StockCode)]['Product Name'])


  print("\n --------- Top 5 Recommendations ------------ \n")

  print(all_predictions[all_predictions['CustomerID']==customer_id].nlargest(5,'probability'))

  recommend = all_predictions[all_predictions['CustomerID']==customer_id].nlargest(5,'probability')

  print('\n-------Product Name of Recommendations ------\n')

  print(df_product[df_product.StockCode.isin(recommend.StockCode)]['Product Name'])


##5. Recommendation's

In [ ]:
recommend_product(13137)

 
---------- Top 5 Bought StockCodes -----------

        CustomerID StockCode  Quantity
234414       13137     84077        48
234443       13137     23321        13
50797        13137     21985        12
234404       13137     22296        12
234418       13137     22297        12

-------Product Name of bought StockCodes ------

70      MightySkins Skin Decal Wrap Compatible with Li...
490           Window Tint Film Mitsubishi (all doors) DIY
694     Harriton Men's Paradise Short-Sleeve Performan...
1065    MightySkins Skin For Samsung Galaxy J3 (2016),...
1339    MightySkins Skin Decal Wrap Compatible with Le...
Name: Product Name, dtype: object

 --------- Top 5 Recommendations ------------ 

         CustomerID  StockCode  probability
1354297       13137      22383     0.957265
1353311       13137      21034     0.954008
1355666       13137      84991     0.936247
1354567       13137      22666     0.902037
1353156       13137      20712     0.894706

-------Product Name of Recom

In [ ]:
recommend_product(15127)


 
---------- Top 5 Bought StockCodes -----------

        CustomerID StockCode  Quantity
272296       15127     23263        48
272287       15127     23354        24
272288       15127     22813        24
272289       15127     23096        24
272285       15127     21181        12

-------Product Name of bought StockCodes ------

13                billyboards Porcelain School Chalkboard
374     MightySkins Protective Vinyl Skin Decal for Po...
923     Zoan Synchrony Duo Sport Electric Snow Helmet ...
952     MightySkins Skin Decal Wrap Compatible with Sm...
1576    EMPIRE KLIX Klutch Designer Wallet Case for LG G2
Name: Product Name, dtype: object

 --------- Top 5 Recommendations ------------ 

         CustomerID  StockCode  probability
4600086       15127      23356     0.800556
4598934       15127      22113     0.798470
4598730       15127      21818     0.795763
4598408       15127      21314     0.786580
4598794       15127      21908     0.785450

-------Product Name of Recom

In [ ]:
recommend_product(15187)

 
---------- Top 5 Bought StockCodes -----------

        CustomerID StockCode  Quantity
168005       15187     84945        72
230220       15187     22128        60
265118       15187     84945        48
127141       15187     22197        36
230218       15187     23310        36

-------Product Name of bought StockCodes ------

236     MightySkins Skin Decal Wrap Compatible with HP...
559     Extreme Networks OUTDOOR IP66 802.3AT GB ETHRT...
1817                     Women's Rialto Adora Ballet Flat
1821    Clear 18 Note Acrylic Box Musical Paperweight ...
Name: Product Name, dtype: object

 --------- Top 5 Recommendations ------------ 

         CustomerID  StockCode  probability
4702533       15187      22333     0.935347
4703203       15187      23041     0.934376
4701767       15187      21240     0.927637
4703424       15187      23286     0.918538
4701765       15187      21238     0.918083

-------Product Name of Recommendations ------

1       Eye Buy Express Prescription Gl

# **Model2 on NueMF**

In [ ]:
model2 = NCF (
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=16,
    layer_sizes=[33,16,8],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

/usr/local/lib/python3.11/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [ ]:
with Timer() as train_time:
    model2.fit(data)

print("Took {} seconds for training.".format(train_time))

Took 1223.9398 seconds for training.


In [ ]:
with Timer() as test_time:
    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item)
        users.extend(user)
        items.extend(item)
        preds.extend(list(model2.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time))

Took 23.3759 seconds for prediction.


In [ ]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.031180
NDCG:	0.072715
Precision@K:	0.049828
Recall@K:	0.050338


In [ ]:
all_predictions[all_predictions['userID']==12346].nlargest(5,'prediction')

,userID,itemID,timestamp,prediction
1916,12346,23165,NaT,0.999757
1918,12346,23167,NaT,0.998666
1725,12346,22960,NaT,0.983642
1993,12346,23243,NaT,0.968019
1777,12346,23014,NaT,0.962905


In [ ]:
all_predictions = all_predictions.rename(columns={

    "userID":'CustomerID',"itemID":'StockCode',"rating":'Quantity','prediction':'probability'

})

In [ ]:
def recommend_product(customer_id):

  print(" \n---------- Top 5 Bought StockCodes -----------\n")


  print(df_order[df_order['CustomerID']==customer_id][['CustomerID','StockCode','Quantity']].nlargest(5,'Quantity'))

  top_5_bought = df_order[df_order['CustomerID']==customer_id][['CustomerID','StockCode','Quantity']].nlargest(5,'Quantity')

  print('\n-------Product Name of bought StockCodes ------\n')

  print(df_product[df_product.StockCode.isin(top_5_bought.StockCode)]['Product Name'])


  print("\n --------- Top 5 Recommendations ------------ \n")

  print(all_predictions[all_predictions['CustomerID']==customer_id].nlargest(5,'probability'))

  recommend = all_predictions[all_predictions['CustomerID']==customer_id].nlargest(5,'probability')

  print('\n-------Product Name of Recommendations ------\n')

  print(df_product[df_product.StockCode.isin(recommend.StockCode)]['Product Name'])


###5. Recommendation's

In [ ]:
recommend_product(13137)

 
---------- Top 5 Bought StockCodes -----------

        CustomerID StockCode  Quantity
234414       13137     84077        48
234443       13137     23321        13
50797        13137     21985        12
234404       13137     22296        12
234418       13137     22297        12

-------Product Name of bought StockCodes ------

70      MightySkins Skin Decal Wrap Compatible with Li...
490           Window Tint Film Mitsubishi (all doors) DIY
694     Harriton Men's Paradise Short-Sleeve Performan...
1065    MightySkins Skin For Samsung Galaxy J3 (2016),...
1339    MightySkins Skin Decal Wrap Compatible with Le...
Name: Product Name, dtype: object

 --------- Top 5 Recommendations ------------ 

         CustomerID  StockCode timestamp  probability
1355666       13137      84991       NaT     0.977292
1354297       13137      22383       NaT     0.963424
1354533       13137      22629       NaT     0.940922
1355209       13137      23322       NaT     0.940029
1353452       13137    

In [ ]:
recommend_product(15127)


 
---------- Top 5 Bought StockCodes -----------

        CustomerID StockCode  Quantity
272296       15127     23263        48
272287       15127     23354        24
272288       15127     22813        24
272289       15127     23096        24
272285       15127     21181        12

-------Product Name of bought StockCodes ------

13                billyboards Porcelain School Chalkboard
374     MightySkins Protective Vinyl Skin Decal for Po...
923     Zoan Synchrony Duo Sport Electric Snow Helmet ...
952     MightySkins Skin Decal Wrap Compatible with Sm...
1576    EMPIRE KLIX Klutch Designer Wallet Case for LG G2
Name: Product Name, dtype: object

 --------- Top 5 Recommendations ------------ 

         CustomerID  StockCode timestamp  probability
4600363       15127      82582       NaT     0.999040
4599505       15127      22737       NaT     0.998897
4599348       15127      22574       NaT     0.996291
4599405       15127      22633       NaT     0.994108
4599404       15127    

In [ ]:
recommend_product(15187)

 
---------- Top 5 Bought StockCodes -----------

        CustomerID StockCode  Quantity
168005       15187     84945        72
230220       15187     22128        60
265118       15187     84945        48
127141       15187     22197        36
230218       15187     23310        36

-------Product Name of bought StockCodes ------

236     MightySkins Skin Decal Wrap Compatible with HP...
559     Extreme Networks OUTDOOR IP66 802.3AT GB ETHRT...
1817                     Women's Rialto Adora Ballet Flat
1821    Clear 18 Note Acrylic Box Musical Paperweight ...
Name: Product Name, dtype: object

 --------- Top 5 Recommendations ------------ 

         CustomerID  StockCode timestamp  probability
4703203       15187      23041       NaT     0.999386
4701734       15187      21195       NaT     0.996247
4702546       15187      22346       NaT     0.995736
4703241       15187      23082       NaT     0.985587
4703432       15187      23294       NaT     0.983911

-------Product Name of Rec

In [ ]:
Model2

# **Model3 on GMF**

In [ ]:
model3 = NCF (
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="GMF",
    n_factors=16,
    layer_sizes=[32,16,8],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

/usr/local/lib/python3.11/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [ ]:
with Timer() as train_time:
    model3.fit(data)

print("Took {} seconds for training.".format(train_time))

Took 1005.2480 seconds for training.


In [ ]:
with Timer() as test_time:
    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item)
        users.extend(user)
        items.extend(item)
        preds.extend(list(model3.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time))

Took 22.8448 seconds for prediction.


In [ ]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.046041
NDCG:	0.095829
Precision@K:	0.061178
Recall@K:	0.078526


In [ ]:
all_predictions[all_predictions['userID']==12346].nlargest(5,'prediction')

,userID,itemID,timestamp,prediction
1500,12346,22727,NaT,0.989875
1726,12346,22961,NaT,0.987679
1401,12346,22624,NaT,0.981595
1499,12346,22726,NaT,0.981425
1918,12346,23167,NaT,0.981104


In [ ]:
all_predictions = all_predictions.rename(columns={

    "userID":'CustomerID',"itemID":'StockCode',"rating":'Quantity','prediction':'probability'

})

In [ ]:
def recommend_product(customer_id):

  print(" \n---------- Top 5 Bought StockCodes -----------\n")


  print(df_order[df_order['CustomerID']==customer_id][['CustomerID','StockCode','Quantity']].nlargest(5,'Quantity'))

  top_5_bought = df_order[df_order['CustomerID']==customer_id][['CustomerID','StockCode','Quantity']].nlargest(5,'Quantity')

  print('\n-------Product Name of bought StockCodes ------\n')

  print(df_product[df_product.StockCode.isin(top_5_bought.StockCode)]['Product Name'])


  print("\n --------- Top 5 Recommendations ------------ \n")

  print(all_predictions[all_predictions['CustomerID']==customer_id].nlargest(5,'probability'))

  recommend = all_predictions[all_predictions['CustomerID']==customer_id].nlargest(5,'probability')

  print('\n-------Product Name of Recommendations ------\n')

  print(df_product[df_product.StockCode.isin(recommend.StockCode)]['Product Name'])


###5. Recommendation's

In [ ]:
recommend_product(13137)

 
---------- Top 5 Bought StockCodes -----------

        CustomerID StockCode  Quantity
234414       13137     84077        48
234443       13137     23321        13
50797        13137     21985        12
234404       13137     22296        12
234418       13137     22297        12

-------Product Name of bought StockCodes ------

70      MightySkins Skin Decal Wrap Compatible with Li...
490           Window Tint Film Mitsubishi (all doors) DIY
694     Harriton Men's Paradise Short-Sleeve Performan...
1065    MightySkins Skin For Samsung Galaxy J3 (2016),...
1339    MightySkins Skin Decal Wrap Compatible with Le...
Name: Product Name, dtype: object

 --------- Top 5 Recommendations ------------ 

         CustomerID  StockCode timestamp  probability
1353883       13137      21900       NaT     0.965388
1355666       13137      84991       NaT     0.962938
1353884       13137      21901       NaT     0.958318
1354297       13137      22383       NaT     0.949752
1353882       13137    

In [ ]:
recommend_product(15127)


 
---------- Top 5 Bought StockCodes -----------

        CustomerID StockCode  Quantity
272296       15127     23263        48
272287       15127     23354        24
272288       15127     22813        24
272289       15127     23096        24
272285       15127     21181        12

-------Product Name of bought StockCodes ------

13                billyboards Porcelain School Chalkboard
374     MightySkins Protective Vinyl Skin Decal for Po...
923     Zoan Synchrony Duo Sport Electric Snow Helmet ...
952     MightySkins Skin Decal Wrap Compatible with Sm...
1576    EMPIRE KLIX Klutch Designer Wallet Case for LG G2
Name: Product Name, dtype: object

 --------- Top 5 Recommendations ------------ 

         CustomerID  StockCode timestamp  probability
4600104       15127      23378       NaT     0.987507
4600079       15127      23349       NaT     0.978675
4600363       15127      82582       NaT     0.975772
4599713       15127      22952       NaT     0.970756
4600086       15127    

In [ ]:
recommend_product(15187)

 
---------- Top 5 Bought StockCodes -----------

        CustomerID StockCode  Quantity
168005       15187     84945        72
230220       15187     22128        60
265118       15187     84945        48
127141       15187     22197        36
230218       15187     23310        36

-------Product Name of bought StockCodes ------

236     MightySkins Skin Decal Wrap Compatible with HP...
559     Extreme Networks OUTDOOR IP66 802.3AT GB ETHRT...
1817                     Women's Rialto Adora Ballet Flat
1821    Clear 18 Note Acrylic Box Musical Paperweight ...
Name: Product Name, dtype: object

 --------- Top 5 Recommendations ------------ 

         CustomerID  StockCode timestamp  probability
4703203       15187      23041       NaT     0.993513
4703424       15187      23286       NaT     0.952153
4703400       15187      23252       NaT     0.948705
4703359       15187      23209       NaT     0.942544
4702778       15187      22600       NaT     0.933662

-------Product Name of Rec

# **Model4 on MLP**

In [ ]:
model4 = NCF (
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="MLP",
    n_factors=16,
    layer_sizes=[32,16,8],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

/usr/local/lib/python3.11/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [ ]:
with Timer() as train_time:
    model4.fit(data)

print("Took {} seconds for training.".format(train_time))

Took 1000.7792 seconds for training.


In [ ]:
with Timer() as test_time:
    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item)
        users.extend(user)
        items.extend(item)
        preds.extend(list(model4.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time))

Took 23.7270 seconds for prediction.


In [ ]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.039539
NDCG:	0.086356
Precision@K:	0.057404
Recall@K:	0.069560


In [ ]:
all_predictions[all_predictions['userID']==12346].nlargest(5,'prediction')

,userID,itemID,timestamp,prediction
1471,12346,22697,NaT,0.983422
1473,12346,22699,NaT,0.982518
1212,12346,22423,NaT,0.978979
1916,12346,23165,NaT,0.978498
1472,12346,22698,NaT,0.974935


In [ ]:
all_predictions = all_predictions.rename(columns={

    "userID":'CustomerID',"itemID":'StockCode',"rating":'Quantity','prediction':'probability'

})

In [ ]:
def recommend_product(customer_id):

  print(" \n---------- Top 5 Bought StockCodes -----------\n")


  print(df_order[df_order['CustomerID']==customer_id][['CustomerID','StockCode','Quantity']].nlargest(5,'Quantity'))

  top_5_bought = df_order[df_order['CustomerID']==customer_id][['CustomerID','StockCode','Quantity']].nlargest(5,'Quantity')

  print('\n-------Product Name of bought StockCodes ------\n')

  print(df_product[df_product.StockCode.isin(top_5_bought.StockCode)]['Product Name'])


  print("\n --------- Top 5 Recommendations ------------ \n")

  print(all_predictions[all_predictions['CustomerID']==customer_id].nlargest(5,'probability'))

  recommend = all_predictions[all_predictions['CustomerID']==customer_id].nlargest(5,'probability')

  print('\n-------Product Name of Recommendations ------\n')

  print(df_product[df_product.StockCode.isin(recommend.StockCode)]['Product Name'])


###5. Recommendation's

In [ ]:
recommend_product(13137)

 
---------- Top 5 Bought StockCodes -----------

        CustomerID StockCode  Quantity
234414       13137     84077        48
234443       13137     23321        13
50797        13137     21985        12
234404       13137     22296        12
234418       13137     22297        12

-------Product Name of bought StockCodes ------

70      MightySkins Skin Decal Wrap Compatible with Li...
490           Window Tint Film Mitsubishi (all doors) DIY
694     Harriton Men's Paradise Short-Sleeve Performan...
1065    MightySkins Skin For Samsung Galaxy J3 (2016),...
1339    MightySkins Skin Decal Wrap Compatible with Le...
Name: Product Name, dtype: object

 --------- Top 5 Recommendations ------------ 

         CustomerID  StockCode timestamp  probability
1354297       13137      22383       NaT     0.962723
1354269       13137      22355       NaT     0.941458
1354567       13137      22666       NaT     0.934066
1353646       13137      21539       NaT     0.923021
1354270       13137    

In [ ]:
recommend_product(15127)


 
---------- Top 5 Bought StockCodes -----------

        CustomerID StockCode  Quantity
272296       15127     23263        48
272287       15127     23354        24
272288       15127     22813        24
272289       15127     23096        24
272285       15127     21181        12

-------Product Name of bought StockCodes ------

13                billyboards Porcelain School Chalkboard
374     MightySkins Protective Vinyl Skin Decal for Po...
923     Zoan Synchrony Duo Sport Electric Snow Helmet ...
952     MightySkins Skin Decal Wrap Compatible with Sm...
1576    EMPIRE KLIX Klutch Designer Wallet Case for LG G2
Name: Product Name, dtype: object

 --------- Top 5 Recommendations ------------ 

         CustomerID  StockCode timestamp  probability
4600002       15127      23265       NaT     0.986723
4600003       15127      23266       NaT     0.977938
4600086       15127      23356       NaT     0.968227
4599958       15127      23213       NaT     0.949266
4599373       15127    

In [ ]:
recommend_product(15187)

 
---------- Top 5 Bought StockCodes -----------

        CustomerID StockCode  Quantity
168005       15187     84945        72
230220       15187     22128        60
265118       15187     84945        48
127141       15187     22197        36
230218       15187     23310        36

-------Product Name of bought StockCodes ------

236     MightySkins Skin Decal Wrap Compatible with HP...
559     Extreme Networks OUTDOOR IP66 802.3AT GB ETHRT...
1817                     Women's Rialto Adora Ballet Flat
1821    Clear 18 Note Acrylic Box Musical Paperweight ...
Name: Product Name, dtype: object

 --------- Top 5 Recommendations ------------ 

         CustomerID  StockCode timestamp  probability
4702581       15187      22386       NaT     0.898579
4701942       15187      21508       NaT     0.861694
4702259       15187      22029       NaT     0.852909
4702890       15187      22716       NaT     0.849429
4703382       15187      23232       NaT     0.839527

-------Product Name of Rec